# RPGゲームを作ろう

### ・セクション1: キャラクターClassを作ろう

In [2]:
#クラスの作成
class Character:
    def __init__(self, name, hp, power):
        self.name = name
        self.hp = hp
        self.power = power
    
    def attack(self, other):
        other.hp -= self.power
        return f"{self.name}が{other.name}を攻撃！{other.name}の体力は{other.hp}になった。"

In [3]:
#インスタンスの生成
hero = Character("勇者", 100, 15)
monster = Character("スライム", 50, 10)

In [4]:
#戦闘
print(hero.attack(monster))
print(monster.hp)

勇者がスライムを攻撃！スライムの体力は35になった。
35


In [5]:
"""
練習問題1) キャラクターにmpを追加しよう。また、attackメソッドを変更して、体力が0より小さくならないようにしよう
"""

class Character:
    def __init__(self, name, hp, power, mp):
        self.name = name
        self.hp = hp
        self.max_hp = hp
        self.power = power
        self.mp = mp
    
    def attack(self, other):
        other.hp -= self.power
        if other.hp <= 0:
            other.hp = 0
            return f"{self.name}が{other.name}を攻撃！{other.name}の体力は{other.hp}になった。\n{other.name}は{self.name}に倒された!"
        return f"{self.name}が{other.name}を攻撃！{other.name}の体力は{other.hp}になった。"

    #生存チェック
    def is_alive(self):
        return self.hp > 0

In [6]:
#インスタンスの生成
hero = Character("勇者", 100, 15, 10)
monster = Character("スライム", 50, 10, 5)

for i in range(4):
    print(hero.attack(monster))

勇者がスライムを攻撃！スライムの体力は35になった。
勇者がスライムを攻撃！スライムの体力は20になった。
勇者がスライムを攻撃！スライムの体力は5になった。
勇者がスライムを攻撃！スライムの体力は0になった。
スライムは勇者に倒された!


---
###  ・セクション2: キャラクターの派生クラスの冒険者クラスを作ろう

In [57]:
class Explorer(Character):
    def __init__(self, name, hp, power, mp, role, level=1):
        super().__init__(name, hp, power, mp)
        self.role = role
        self.level = level
        self._initial_state = copy.deepcopy(self.__dict__)

    #スペシャルアタックを追加
    def special_attack(self, other):
        if self.mp < 3:
            return f"MPが足りない！ (現在のMP{self.mp})"          
        #勇者の必殺技
        if self.role == HERO:
            if not isinstance(other, Explorer):
                other.hp -= self.power*2
                if self.hp <= self.max_hp * 0.4:
                    other.hp = self.power*2.7
                self.mp -= 3  
                if other.hp <= 0:
                    other.hp = 0
                    return f"{self.name}の閃空斬り発動！{other.name}の体力は{other.hp}になった。\n{other.name}は{self.name}に倒された!"
                return f"{self.name}の閃空斬り発動！{other.name}の体力は{other.hp}になった。"
            return f"{self.name}は味方に必殺技を使えない！"
            
        #ヒーラーの必殺技
        elif self.role == HEALER:
            if isinstance(other, Explorer):
                other.hp += other.max_hp * 0.30
                self.mp -= 3
                return f"{self.name}の女神の加護発動！{other.name}の体力は{other.hp}になった。"
            return f"{self.name}は敵を回復できない！"
            
        #魔法使いの必殺技
        elif self.role == MAGICIAN:
            if isinstance(other, Explorer):
                other.mp += 4
                self.mp -= 3
                return f"{self.name}のマジックゲイン発動！{other.name}のMPは{other.mp}になった。"
            return f"{self.name}は敵にMP回復を施せない！"
        
        return f"{self.name}: 不明な役割のため、何も起こらなかった。"

    def reset(self):
        """初期状態に戻す"""
        self.__dict__ = copy.deepcopy(self._initial_state)

In [60]:
""" 
練習問題2) モンスタークラスを作成しよう。DORAGON、HARPY、WITCHの3タイプの必殺技を作ろう
"""

class Monster(Character):
    def __init__(self, name, hp, power, mp, breed):
        super().__init__(name, hp, power, mp)
        self.breed = breed
        self._initial_state = copy.deepcopy(self.__dict__)

    #スペシャルアタック
    def special_attack(self, other):
        if self.mp < 3:
            return f"MPが足りない！ (現在のMP{self.mp})"

        #ドラゴンの必殺技
        if self.breed == DRAGON:
            #自分の攻撃力の2.5倍のダメージを与える
            other.hp -= self.power*2.5
            self.mp -= 3
            if other.hp <= 0:
                other.hp = 0
                return f"{self.name}のファイヤーブレス発動！{other.name}の体力は{other.hp}になった。\n{other.name}は{self.name}に倒された!"
            return f"{self.name}のファイヤーブレス発動！{other.name}の体力は{other.hp}になった。"
            
        #ハーピーの必殺技
        elif self.breed== HARPY:
            #自分の現在のhpの50パーセントのダメージを与え、自分のmax体力の50%回復
            other.hp -= self.hp*0.5
            self.hp += self.max_hp*0.5
            self.mp -= 3
            return f"{self.name}の再生の歌声発動！{other.name}の体力が{other.hp}になった。{self.name}はHP{self.hp}に回復した。"

        #ウィッチの必殺技
        elif self.breed == WITCH:
            #相手のmp2と体力の20%を奪い取って回復
            other.mp -= other.mp - 2
            if other.mp < 0:
                other.mp = 0
            other.hp = other.hp * 0.80
            self.hp += other.hp * 0.80
            self.mp -= 3
            return f"{self.name}のドレイン発動！{other.name}のMPが{other.mp}、体力が{other.hp}になった。"

In [61]:
# --- ゲーム本体 (バトルマネージャー) --- 
class BattleManager:
    def __init__(self, party, enemy_team):
        self.party = party
        self.enemy_team = enemy_team
        self.turn = 1
        self.is_game_over = False

    def display_status(self):
        #全員のステータスを表示
        print("\n" + "="*40)
        print(f"--- ターン: {self.turn} ---")
        print("\n[勇者パーティ]")
        for i, member in enumerate(self.party):
            print(f" {i+1}. {member.name} ({member.role}): HP {member.hp}/{member.max_hp}, MP {member.mp}")
        
        print("\n[モンスター]")
        # 1体のみ表示される
        for i, monster in enumerate(self.enemy_team):
            print(f" {i+1}. {monster.name} ({monster.breed}): HP {monster.hp}/{monster.max_hp}, MP {monster.mp}")
        print("="*40 + "\n")

    def player_turn(self):
        #プレイヤーが操作するターン
        print("--- あなたのターン ---")
        
        for player in self.party:
            if not player.is_alive():
                continue

            print(f"\n[{player.name} の行動]")
            
            def select_target(targets, target_type):
                while True:
                    try:
                        print(f"ターゲットを選択してください:")
                        # 生きているターゲットのみ表示
                        alive_targets = [t for t in targets if t.is_alive()]
                        if not alive_targets:
                             print("ターゲットがいません！")
                             return None

                        for i, target in enumerate(alive_targets):
                            print(f" {i+1}. {target.name} (HP: {target.hp} MP: {target.mp})")
                        
                        target_index = int(input("番号を入力 (1/...): ")) - 1
                        
                        if 0 <= target_index < len(alive_targets):
                            return alive_targets[target_index]
                        else:
                            print("無効な番号です。再入力してください。")
                    except ValueError:
                        print("数字を入力してください。")
            
            # ターゲットが存在しない場合はスキップ
            if not self.enemy_team and (player.role == HERO or player.role == MAGICIAN):
                 continue
            if not self.party and (player.role == HEALER or player.role == MAGICIAN):
                 continue

            while True:
                action = input(f"行動を選択 (a:通常攻撃 / s:必殺技): ").lower()
                
                if action == 'a':
                    target = select_target(self.enemy_team, "モンスター")
                    if target:
                        print(player.attack(target))
                        break
                
                elif action == 's':
                    if player.role == HERO:
                        target = select_target(self.enemy_team, "モンスター")
                    elif player.role == HEALER or player.role == MAGICIAN:
                        target = select_target(self.party, "パーティメンバー")
                    else:
                        print("その役割には必殺技がありません。")
                        continue
                    
                    if target:
                        print(player.special_attack(target))
                        break
                
                else:
                    print("無効な入力です。'a' または 's' を入力してください。")
            
            if self.check_game_status():
                return

    def enemy_turn(self):
        #モンスターが行動するターン
        print("\n--- 相手のターン ---")
        
        alive_players = [p for p in self.party if p.is_alive()]
        if not alive_players:
            return

        for monster in self.enemy_team:
            if not monster.is_alive():
                continue

            target = random.choice(alive_players)

            
            # モンスターのAI: 必殺技 or 通常攻撃
            if random.random() < 0.5 and monster.mp >= 3: #or \
               #(monster.mp >= 3 and monster.hp < monster.max_hp * 0.5): # HPが低いと確定で反撃
                 print(monster.special_attack(target))
            else:
                print(monster.attack(target))

            if self.check_game_status():
                return

    def check_game_status(self):
        #勝敗のチェック
        alive_party = [p for p in self.party if p.is_alive()]
        alive_enemies = [m for m in self.enemy_team if m.is_alive()]
        
        if not alive_party:
            print("\n!!! 勇者パーティは全滅した。ゲームオーバー !!!")
            self.is_game_over = True
            return True
        
        if not alive_enemies:
            print("\n!!! モンスターをすべて倒した！勝利 !!!")
            self.is_game_over = True
            return True
        
        self.party = alive_party
        self.enemy_team = alive_enemies
        return False

    def start_battle(self):
        #戦闘の開始と実行
        enemy_name = self.enemy_team[0].name if self.enemy_team else "敵"
        print(f"{enemy_name}が立ちはだかる！バトル開始！")
        while not self.is_game_over:
            self.display_status()
            
            self.player_turn()
            if self.is_game_over: break
            
            self.enemy_turn()
            if self.is_game_over: break
            
            self.turn += 1
        
        print("\n--- 戦闘終了 ---")

In [62]:
import copy
import random
import sys

#Explorerの役割とMonsterの種族の決定
HERO = 'hero'
HEALER = 'healer'
MAGICIAN = 'magician'
DRAGON = 'dragon'
HARPY = 'harpy'
WITCH = 'witch'

if __name__ == "__main__":
    # 勇者パーティーの生成
    party_members = [
        Explorer("アーサー", 100, 10, 6, HERO),
        Explorer("リア", 90, 9, 9, HEALER),
        Explorer("マーリン", 80, 12, 12, MAGICIAN),
    ]

    # --- モンスター隊の生成 (ランダムに1体を選択) ---
    MONSTER_TEMPLATES = {
        DRAGON: ("ファイヤードレイク", 200, 25, 6),  # HP高め、高攻撃力
        HARPY: ("セイレーン", 150, 20, 9),   # HP中、回復持ち
        WITCH: ("リリス", 180, 23, 12),   # HP中、MPドレイン持ち
    }

    # 3つの種族からランダムに1つ選ぶ
    random_breed = random.choice([DRAGON, HARPY, WITCH])
    name, hp, power, mp = MONSTER_TEMPLATES[random_breed]
    
    # 選択された1体のみを生成
    single_monster = Monster(name, hp, power, mp, random_breed)
    enemy_squad = [single_monster]
    
    # バトル開始
    try:
        game = BattleManager(party_members, enemy_squad)
        game.start_battle()
    except EOFError:
        print("\nゲームを終了します。")
    except Exception as e:
        print(f"\n予期せぬエラーが発生しました: {e}")
        sys.exit(1)

セイレーンが立ちはだかる！バトル開始！

--- ターン: 1 ---

[勇者パーティ]
 1. アーサー (hero): HP 100/100, MP 6
 2. リア (healer): HP 90/90, MP 9
 3. マーリン (magician): HP 80/80, MP 12

[モンスター]
 1. セイレーン (harpy): HP 150/150, MP 9

--- あなたのターン ---

[アーサー の行動]


行動を選択 (a:通常攻撃 / s:必殺技):  s


ターゲットを選択してください:
 1. セイレーン (HP: 150 MP: 9)


番号を入力 (1/...):  1


アーサーの閃空斬り発動！セイレーンの体力は130になった。

[リア の行動]


...

マーリンがセイレーンを攻撃！セイレーンの体力は8.0になった。

--- 相手のターン ---
セイレーンがマーリンを攻撃！マーリンの体力は0になった。
マーリンはセイレーンに倒された!

--- ターン: 13 ---

[勇者パーティ]
 1. アーサー (hero): HP 60/100, MP 1
 2. リア (healer): HP 21.0/90, MP 0

[モンスター]
 1. セイレーン (harpy): HP 8.0/150, MP 0

--- あなたのターン ---

[アーサー の行動]


行動を選択 (a:通常攻撃 / s:必殺技):  a


ターゲットを選択してください:
 1. セイレーン (HP: 8.0 MP: 0)


番号を入力 (1/...):  1


アーサーがセイレーンを攻撃！セイレーンの体力は0になった。
セイレーンはアーサーに倒された!

!!! モンスターをすべて倒した！勝利 !!!

--- 戦闘終了 ---
